# Computing components of the Einstein tensor symbolically using sympy.tensor

In [1]:
from sympy.tensor.tensor import TensorIndexType, TensorHead, tensor_indices
from sympy.tensor.toperators import PartialDerivative
import sympy as sp

In [2]:
# Define tensor indices
Lorentz = TensorIndexType("Lorentz", dummy_name=r"\lambda")
mu, nu, rho, sigma, alpha, beta = tensor_indices(r"\mu \nu \rho \sigma \alpha \beta", Lorentz)

In [3]:
# Tensors are defined by their name and type of indices
x = TensorHead("x", [Lorentz]) # coordinates
g = TensorHead("g", [Lorentz,Lorentz]) # metric
Gamma = TensorHead(r"\Gamma", [Lorentz,Lorentz,Lorentz]) # Christoffel symbols
Riemann_tensor = TensorHead(r"R", [Lorentz,Lorentz,Lorentz,Lorentz]) # Riemann tensor
Ricci_tensor = TensorHead(r"R", [Lorentz,Lorentz]) # Ricci tensor

In [4]:
Christoffel_expr = sp.Rational(1,2) * g(rho,sigma)* (
     PartialDerivative(g(-sigma,-mu), x(nu))
    +PartialDerivative(g(-sigma,-nu), x(mu))
    -PartialDerivative(g(-mu,-nu), x(sigma)))

# Riemann tensor expression in terms of Gamma
Riemann_expr = (
    PartialDerivative(Gamma(alpha, -mu, -nu), x(beta))
  - PartialDerivative(Gamma(alpha, -mu, -beta), x(nu))
  + Gamma(alpha, -beta, -rho) * Gamma(rho, -mu, -nu)
  - Gamma(alpha, -nu, -rho) * Gamma(rho, -mu, -beta)
)

Ricci_expr = (
    g(-alpha,beta) * Riemann_tensor(alpha, -mu, -beta, -nu)
)

display(sp.Eq(Gamma(rho,-mu,-nu),Christoffel_expr))
display(sp.Eq(Riemann_tensor(alpha,-mu,-beta,-nu),Riemann_expr))
display(sp.Eq(Ricci_tensor(-mu,-nu),Ricci_expr))

Eq(\Gamma(\rho, -\mu, -\nu), (1/2)*g(\rho, \lambda_0)*(PartialDerivative(g(-\lambda_0, -\mu), x(\nu)) + PartialDerivative(g(-\lambda_0, -\nu), x(\mu)) + (-1)*PartialDerivative(g(-\mu, -\nu), x(\lambda_0))))

Eq(R(\alpha, -\mu, -\beta, -\nu), PartialDerivative(\Gamma(\alpha, -\mu, -\nu), x(\beta)) - PartialDerivative(\Gamma(\alpha, -\mu, -\beta), x(\nu)) + \Gamma(\alpha, -\beta, -\lambda_0)*\Gamma(\lambda_0, -\mu, -\nu) - \Gamma(\alpha, -\nu, -\lambda_0)*\Gamma(\lambda_0, -\mu, -\beta))

Eq(R(-\mu, -\nu), g(-\lambda_1, \lambda_0)*R(\lambda_1, -\mu, -\lambda_0, -\nu))

# Substitute in explicit metric and coordinates

In [5]:
# Coordinates
t = sp.symbols("t")
r = sp.symbols("r", positive=True)
theta, phi = sp.symbols("theta phi")
coords_array = [t,r,theta,phi]

# Define the metric
Q = sp.symbols("Q") # gravitational charge parameter 
metric_array = [[-(1-(2*Q)/r),0,0,0],
                 [0,1/(1-(2*Q)/r),0,0],
                 [0,0,r**2,0],
                 [0,0,0,r**2 * sp.sin(theta)**2]]

In [6]:
running_repl = {
    g(-mu,-nu): metric_array,
    x(mu): coords_array,
    Lorentz: metric_array
}

### Compute the components

In [7]:
# Compute Christoffel as symbolic array
christ_array = Christoffel_expr.replace_with_arrays(running_repl)
running_repl[Gamma(mu,-nu,-rho)] = christ_array

In [8]:
# Compute Riemann and Ricci as symbolic arrays
Riemann_array = Riemann_expr.replace_with_arrays(running_repl)
running_repl[Riemann_tensor(mu,-nu,-rho,-sigma)] = Riemann_array

In [9]:
sp.simplify(Riemann_array)

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 2*Q/(r**2*(2*Q - r)), 0, 0], [2*Q/(r**2*(-2*Q + r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, Q/r, 0], [0, 0, 0, 0], [-Q/r, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, Q*sin(theta)**2/r], [0, 0, 0, 0], [0, 0, 0, 0], [-Q*sin(theta)**2/r, 0, 0, 0]]], [[[0, 2*Q*(2*Q - r)/r**4, 0, 0], [2*Q*(-2*Q + r)/r**4, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, Q/r, 0], [0, -Q/r, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, Q*sin(theta)**2/r], [0, 0, 0, 0], [0, -Q*sin(theta)**2/r, 0, 0]]], [[[0, 0, Q*(-2*Q + r)/r**4, 0], [0, 0, 0, 0], [Q*(2*Q - r)/r**4, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, Q/(r**2*(2*Q - r)), 0], [0, Q/(r**2*(-2*Q + r)), 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, -2*Q*sin(theta)**2/r], [0, 0, 2*Q*sin(theta)**2/r, 0]]], [[[0, 0, 0, Q*(-2*Q + r)/r**4], [0, 0, 0, 0]

In [10]:
ricci_tensor_array = Ricci_expr.replace_with_arrays(running_repl)

In [11]:
sp.simplify(ricci_tensor_array)

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]